In [ ]:
!pip install wandb --upgrad 
!git clone https://github.com/zapatacomputing/z-quantum-core.git
!git clone https://github.com/zapatacomputing/z-quantum-optimizers.git
!git clone https://github.com/zapatacomputing/qe-qiskit.git
!pip install z-quantum-core/ --user
!pip install z-quantum-optimizers/ --user
!pip install qe-qiskit/ --user
!pip install cirq
!pip install wandb


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgrad (--upgrade, --upgrade-strategy?)
fatal: destination path 'z-quantum-core' already exists and is not an empty directory.
fatal: destination path 'z-quantum-optimizers' already exists and is not an empty directory.
fatal: destination path 'qe-qiskit' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./z-quantum-core
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior befor

In [ ]:
import numpy as np
import cirq

def frobenius_norm(matrix_A, matrix_B):
    return np.linalg.norm(matrix_A - matrix_B)


def get_target_unitary(parameters):
    qubits = [cirq.LineQubit(i) for i in range(2)]
    circuit = cirq.Circuit()

    circuit.append(cirq.ry(parameters[0]).on(qubits[0]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))
    circuit.append(cirq.ry(parameters[1]).on(qubits[1]))
    return circuit.unitary()


def get_overparameterized_unitary(parameters):
    qubits = [cirq.LineQubit(i) for i in range(2)]
    circuit = cirq.Circuit()

    circuit.append(cirq.ry(parameters[0]).on(qubits[0]))
    circuit.append(cirq.ry(parameters[1]).on(qubits[1]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))
    circuit.append(cirq.ry(parameters[2]).on(qubits[0]))
    circuit.append(cirq.ry(parameters[3]).on(qubits[1]))
    circuit.append(cirq.rz(parameters[4]).on(qubits[0]))
    circuit.append(cirq.rz(parameters[5]).on(qubits[1]))
    circuit.append(cirq.XXPowGate(exponent=parameters[6]).on(qubits[0], qubits[1]))
    circuit.append(cirq.ry(parameters[7]).on(qubits[0]))
    circuit.append(cirq.ry(parameters[8]).on(qubits[1]))
    circuit.append(cirq.rz(parameters[9]).on(qubits[0]))
    circuit.append(cirq.rz(parameters[10]).on(qubits[1]))

    return circuit.unitary()

In [ ]:
import wandb
wandb.login()
PROJECT = "QLT"

wandb: Currently logged in as: simonwa7. Use `wandb login --relogin` to force relogin


In [ ]:
from zquantum.optimizers import ScipyOptimizer

SEED = 1234
ALL_UNPRUNED_RESULTS = []
ALL_PRUNED_RESULTS = []
ALL_RANDOMIZED_PRUNED_RESULTS = []

np.random.seed(SEED)
target_parameters = np.random.uniform(0, 2 * np.pi, 2)
target = get_target_unitary(target_parameters)
first_trivial_target_parameters_for_overparameterized_unitary = [target_parameters[0], 0, 0, target_parameters[1], 0, 0, 0, 0, 0, 0, 0]
second_trivial_target_parameters_for_overparameterized_unitary = [target_parameters[0], 0, 0, 0, 0, 0, 0, 0, target_parameters[1], 0, 0]

optimizer = ScipyOptimizer(method="L-BFGS-B", options={'ftol': 1e-6})

def get_parameter_indices_to_be_pruned(parameters):
    indices = []
    for index, parameter in enumerate(parameters):
        if np.isclose(0., parameter%2, atol=1e-3):
          indices.append(index)
        elif np.isclose(2., parameter%2, atol=1e-3):
          indices.append(index)
    return indices

def optimize_unpruned_cost_function(trial_number, unpruned_initial_parameters):

    min_norm = np.inf
    def unpruned_cost_function(parameters):
        global target, first_trivial_target_parameters_for_overparameterized_unitary, second_trivial_target_parameters_for_overparameterized_unitary
        nonlocal min_norm

        norm = frobenius_norm(target, get_overparameterized_unitary(parameters))
        min_norm = min(min_norm, norm)
        log_dict = {
            "Frobenius Norm": norm,
            "Minimum Frobenius Norm": min_norm,
            "Parameter Distance from First Trivial": np.linalg.norm(parameters - first_trivial_target_parameters_for_overparameterized_unitary),
            "Parameter Distance from Second Trivial": np.linalg.norm(parameters - second_trivial_target_parameters_for_overparameterized_unitary),
        }
        wandb.log(log_dict)

        return norm

    run = wandb.init(project=PROJECT, config={
        "optimizer":"SPSA",
        "tolx": 1e-6, 
        "target_parameters": target_parameters,
        "trial": trial_number,
        "type": "unpruned",
        "unpruned_initial_parameters": initial_parameters,
        "initial_parameters": initial_parameters,
        "wandb_nb":PROJECT+"-collab"
    })

    results = optimizer.minimize(unpruned_cost_function, initial_parameters, keep_history=True)
    run.finish()
    
    return results

def optimize_pruned_cost_function(trial_number, unpruned_initial_parameters, pruned_parameter_indices):
    
    min_norm = np.inf
    def pruned_cost_function(parameters):
        global target, pruned_parameter_indices
        nonlocal min_norm

        for pruned_index in pruned_parameter_indices:
            parameters = np.concatenate((parameters[:pruned_index], np.asarray([0]), parameters[pruned_index:]))

        norm = frobenius_norm(target, get_overparameterized_unitary(parameters))
        min_norm = min(min_norm, norm)
        log_dict = {
            "Frobenius Norm": norm,
            "Minimum Frobenius Norm": min_norm,
        }
        wandb.log(log_dict)

        return norm

    pruned_initial_parameters = unpruned_initial_parameters
    for pruned_index in pruned_parameter_indices[::-1]:
        pruned_initial_parameters = np.delete(pruned_initial_parameters, pruned_index)

    run = wandb.init(project=PROJECT, config={
        "optimizer":"SPSA",
        "tolx": 1e-6, 
        "target_parameters": target_parameters,
        "trial": trial_number,
        "type": "pruned",
        "unpruned_initial_parameters": unpruned_initial_parameters,
        "initial_parameters": pruned_initial_parameters,
        "pruned_parameter_indices": pruned_parameter_indices,
        "wandb_nb":PROJECT+"-collab"
    })

    results = optimizer.minimize(pruned_cost_function, pruned_initial_parameters, keep_history=True)
    run.finish()

    return results

def optimize_randomized_pruned_cost_function(trial_number, unpruned_initial_parameters, pruned_parameter_indices):
    
    min_norm = np.inf
    def pruned_cost_function(parameters):
        global target, pruned_parameter_indices
        nonlocal min_norm

        for pruned_index in pruned_parameter_indices:
            parameters = np.concatenate((parameters[:pruned_index], np.asarray([0]), parameters[pruned_index:]))

        norm = frobenius_norm(target, get_overparameterized_unitary(parameters))
        min_norm = min(min_norm, norm)
        log_dict = {
            "Frobenius Norm": norm,
            "Minimum Frobenius Norm": min_norm,
        }
        wandb.log(log_dict)

        return norm

    randomized_pruned_initial_parameters = np.random.uniform(0, 2*np.pi, len(unpruned_initial_parameters))
    for pruned_index in pruned_parameter_indices[::-1]:
        randomized_pruned_initial_parameters = np.delete(randomized_pruned_initial_parameters, pruned_index)

    run = wandb.init(project=PROJECT, config={
        "optimizer":"SPSA",
        "tolx": 1e-6, 
        "target_parameters": target_parameters,
        "trial": trial_number,
        "type": "pruned_and_randomized",
        "unpruned_initial_parameters": unpruned_initial_parameters,
        "initial_parameters": randomized_pruned_initial_parameters,
        "pruned_parameter_indices": pruned_parameter_indices,
        "wandb_nb":PROJECT+"-collab"
    })

    results = optimizer.minimize(pruned_cost_function, randomized_pruned_initial_parameters, keep_history=True)
    run.finish()

    return results

In [ ]:
for trial in range(10, 20):
    min_norm = np.inf
    np.random.seed(SEED+trial)
    initial_parameters = np.random.uniform(0, 2 * np.pi, 11)

    ALL_UNPRUNED_RESULTS.append(optimize_unpruned_cost_function(trial, initial_parameters))

    pruned_parameter_indices = get_parameter_indices_to_be_pruned(ALL_UNPRUNED_RESULTS[-1].opt_params)

    ALL_PRUNED_RESULTS.append(optimize_pruned_cost_function(trial, initial_parameters, pruned_parameter_indices))
    ALL_RANDOMIZED_PRUNED_RESULTS.append(optimize_randomized_pruned_cost_function(trial, initial_parameters, pruned_parameter_indices))

Frobenius Norm,█▇▅█▇▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▇▅▅▅▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Parameter Distance from First Trivial,▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Parameter Distance from Second Trivial,▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0
Parameter Distance from First Trivial,10.7259
Parameter Distance from Second Trivial,11.0575


Frobenius Norm,█▆▂█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▆▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


Frobenius Norm,██▄█▃▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▆▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


Frobenius Norm,█▅▆▇▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▅▅▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Parameter Distance from First Trivial,▁▇█▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Parameter Distance from Second Trivial,▁▇█▄▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0
Parameter Distance from First Trivial,19.09859
Parameter Distance from Second Trivial,14.9464


Frobenius Norm,█▅▆▇▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▅▅▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


...

Frobenius Norm,█▆▄▇▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▆▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


Frobenius Norm,█▆▆▇▃█▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▆▆▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


Frobenius Norm,▇██▇▇█▄▄▆▃▄▄▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,██████▅▅▅▄▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Parameter Distance from First Trivial,▁▅▅▆▆█▅▆▅▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Parameter Distance from Second Trivial,▁▄▄▅▆█▄▅▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0
Parameter Distance from First Trivial,16.62744
Parameter Distance from Second Trivial,16.11678


Frobenius Norm,██▆▆▄▃▂▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▇▆▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


Frobenius Norm,██▆▅█▄▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Minimum Frobenius Norm,█▇▆▅▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Frobenius Norm,0.0
Minimum Frobenius Norm,0.0


In [ ]:
# for i, unpruned_results in enumerate(ALL_UNPRUNED_RESULTS):
#     print("-------TRIAL {}-------".format(i))
#     min_norm = np.inf
#     pruned_parameter_indices = parameter_indices_to_be_pruned(unpruned_results.opt_params)
#     print(pruned_parameter_indices)
#     pruned_initial_parameters = unpruned_results.history[0].params
#     for pruned_index in pruned_parameter_indices[::-1]:
#         pruned_initial_parameters = np.delete(pruned_initial_parameters, pruned_index)
    
#     def pruned_cost_function(parameters):
#         global target, min_norm, pruned_parameter_indices

#         for pruned_index in pruned_parameter_indices:
#             parameters = np.concatenate((parameters[:pruned_index], np.asarray([0]), parameters[pruned_index:]))

#         norm = frobenius_norm(target, get_overparameterized_unitary(parameters))

#         return norm

#     pruned_final_parameters = unpruned_results.history[-1].params
#     for pruned_index in pruned_parameter_indices[::-1]:
#         pruned_final_parameters = np.delete(pruned_final_parameters, pruned_index)
#     pruned_cost = pruned_cost_function(pruned_final_parameters)
#     unpruned_cost = unpruned_results.opt_value
#     if not np.isclose(pruned_cost, unpruned_cost, atol=1e-3):
#       import pdb
#       pdb.set_trace()